In [1]:
# import necessary libraries
import warnings
warnings.filterwarnings("ignore")
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random
from keras.models import load_model

# create an object of WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# importing the GL Bot corpus file for preprocessing
words = []
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open ("data.json").read()
intents = json.loads(data_file)

In [2]:
# preprocessing the json data
# tokenization
import nltk
nltk.download('punkt')
nltk.download('wordnet')
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        #add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print(len(documents), "documents")

# classes = intents
print(len(classes), "classes", classes)

#words = all words, vocabulary
print(len(words), "unique lemmatized words", words)

#creating a pickle file to store the Python objects which we will use while predicting
pickle.dump(words,open('text.pkl','wb'))
pickle.dump(classes,open('label.pkl','wb'))

36 documents
36 classes ['akun', 'akun1', 'akun10', 'akun2', 'akun3', 'akun4', 'akun5', 'akun6', 'akun7', 'akun8', 'akun9', 'antrian', 'antrian1', 'antrian10', 'antrian2', 'antrian3', 'antrian4', 'antrian5', 'antrian6', 'antrian7', 'antrian8', 'antrian9', 'greeting', 'menu', 'nonanswer', 'umum', 'umum1', 'umum10', 'umum2', 'umum3', 'umum4', 'umum5', 'umum6', 'umum7', 'umum8', 'umum9']
91 unique lemmatized words ['ada', 'akun', 'aman', 'anggota', 'antrian', 'apa', 'apakah', 'awal', 'bagaimana', 'bantuan', 'baru', 'berapa', 'biaya', 'bisa', 'bisakah', 'cara', 'dalam', 'dari', 'data', 'datang', 'dengan', 'di', 'diakses', 'dibuat', 'dilakukan', 'dokter', 'e-medika', 'email', 'formulir', 'harus', 'informasi', 'ini', 'itu', 'jadwal', 'jika', 'kata', 'keluarga', 'lakukan', 'lebih', 'lupa', 'masalah', 'medium', 'melalui', 'melihat', 'membatalkan', 'memilih', 'menangani', 'mendaftar', 'mendapatkan', 'mengakses', 'mengalami', 'mengatur', 'menggunakan', 'menghubungi', 'mengisi', 'mengubah', 'menu

In [4]:
# create our training data
training = []

# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    #initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]

    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    #output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

# shuffle features and converting it into numpy arrays
random.shuffle(training)
training = np.array(training, dtype=object)

# create train and test lists
train_x = np.array(list(training[:,0]))
train_y = np.array(list(training[:,1]))

print("Training data created")

Training data created


In [5]:
# create NN model to predict the responses
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('model.h5', hist) # we will pickle this model to use in the future
print("\n")
print("*"*50)
print("\nModel Created Successfully!")

Epoch 1/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.0416 - loss: 3.6964   
Epoch 2/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0305 - loss: 3.5224     
Epoch 3/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0131 - loss: 3.5483     
Epoch 4/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0480 - loss: 3.5075     
Epoch 5/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0640 - loss: 3.4738     
Epoch 6/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0831 - loss: 3.4111     
Epoch 7/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1489 - loss: 3.2889 
Epoch 8/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1904 - loss: 3.3380 
Epoch 9/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1664 - loss: 3.3323 
Epoch 10/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2125 - loss: 3.1468 
Epoch 11/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1644 - loss: 3.0112 
Epoch 12/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - 



**************************************************

Model Created Successfully!
